In [10]:
import torch
from torch.utils.serialization import load_lua
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import re


def code2char(code, vocab):
    vocab_map = dict(zip(vocab.values(), vocab.keys()))
    return "".join(vocab_map[c] for c in code)

def char2code(text, vocab):
    data = torch.ByteTensor(len(text))
    for i, c in enumerate(text):
        data[i] = vocab[c]
    return data


class CharDataset(Dataset):
    """
        Charge un fichier data_file tenseur 1D d'entiers et le decoupe en sequences de longueur seq_length. Vocab_file est un dictionnaire
        de characteres vers entier.
    """
    def __init__(self, data_file, vocab_file, seq_length_entry, seq_length_exit):
        self.data_file = data_file
        self.vocab_file = vocab_file
        self.seq_length_entry = seq_length_entry
        self.seq_length_exit = seq_length_exit
        self.data = torch.load(data_file)
        self.vocab = torch.load(vocab_file)
        self.vocab_map = dict(zip(self.vocab.values(), self.vocab.keys()))
        self.nb_samples = len(self.data) - self.seq_length_exit - self.seq_length_entry +1
        #print('cutting off end of data so that the batches/sequences divide evenly')
        #self.data = self.data[:(self.seq_length_exit * self.nb_samples + 1)]
        self.vocab_size = len(self.vocab)

    def __getitem__(self, index):
        #index = min(index, len(self.data) - self.seq_length_exit - self.seq_length_entry )
        start, end = index, index +  self.seq_length_entry
        return self.data[start:end], self.data[end:(end + self.seq_length_exit)]

    def __len__(self):
        return self.nb_samples



def make_files(text_file, out_tensorfile=None, out_vocabfile=None, vocab_file=None):
    """ Permet de creer a partir d'un fichier texte le tenseur 1D encode en entier et le mapping
    des caracteres vers leur code.
    * text_file : le fichier texte brut
    * out_tensorfile : si specifie, sauve le tenseur 1D de sortie
    * out_vocabfile : si specifie, sauve le mapping entre caractere et code
    * vocab_file : si specifie, charge le mapping entre caractere et code
    """
    #with open(text_file) as f:
    #    text = re.sub(r'[^a-zA-Z0-9_\s]', ' ', f.read().lower())
    #    text = re.sub(r'\s+', ' ', text)

    text = re.split('\W+', text_file)

    if vocab_file is not None:
        vocab = torch.load(vocab_file)
    else:
        chars = set(text)
        chars.add(' ')
        chars.add('.')
        chars.add(',')
        vocab = dict(zip(sorted(chars), range(len(chars))))
    data = char2code(text, vocab)
    if out_vocabfile is not None:
        torch.save(vocab, out_vocabfile)
    if out_tensorfile is not None:
        torch.save(data, out_tensorfile)
    return data, vocab


In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import datasets, transforms
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random
import time
import math
import numpy.random as rdn


class Neural_net(nn.Module):
    def __init__(self, num_layers, size_list, f):
        super(Neural_net, self).__init__()

        self.num_layers = num_layers

        self.linear = nn.ModuleList([nn.Linear(size_list[i], size_list[i + 1]) for i in range(num_layers)])

        self.f = f

    def forward(self, x):
        for layer in range(self.num_layers):
            x = self.f(self.linear[layer](x))

        return x;


class Encoder(nn.Module):
    def __init__(self, size_h, size_dict, f):
        super(Encoder,self).__init__()
        self.w1 = nn.Linear(size_dict, size_h)
        self.w2 = nn.Linear(size_h, size_h)
        self.f = f

    def forward(self, h, x_i):
        result = self.f(self.w1(x_i) + self.w2(h))
        return result;


class Decoder(nn.Module):
    def __init__(self, size_h,size_dict):
        super(Decoder,self).__init__()
        self.w1 = nn.Linear(size_h, size_dict)

    def forward(self, h):
        result = F.softmax(self.w1(h))
        return result;


class RNN(nn.Module):
    def __init__(self,size_x,size_y,size_dict,size_h,f):
        super(RNN,self).__init__()
        self.size_x = size_x
        self.size_y = size_y
        self.size_h = size_h
        self.size_dict = size_dict
        self.f = f

        self.e1 = Encoder(self.size_h, self.size_dict, self.f)
        self.e2 = Encoder(self.size_h, self.size_dict, self.f)
        self.d = Decoder(self.size_h, self.size_dict)

    def forward(self,h,mat_xv,mat_yv):
        out = []
        for i in range(self.size_x):
            h = self.e1.forward(h.view(-1), mat_xv[:, i])
        for j in range(self.size_y):
            out.append(self.d.forward(h.view(-1)))
            h = self.e2.forward(h.view(-1), mat_yv[:, j])
        return out;

    def forward2(self,h,mat_xv):
        out2 = []
        for i in range(self.size_x):
            h = self.e1.forward(h.view(-1), mat_xv[:, i])
        for j in range(self.size_y):
            proba = self.d.forward(h.view(-1))
            pred= torch.multinomial(proba, 1, replacement=True)[0] 
            out2.append(pred.data.numpy()[0])
            #tirage par rapport à
            h = self.e2.forward(h.view(-1), proba)
        return out2;

    
    

In [12]:

import torch
from torchvision import datasets, transforms
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random
import time
import numpy.random as rdn
import re

#PARAMETERS
T=4 # taille d'entrée
size_h = 55 # taille de l'espace latent
batch_size=1
n=3 #taille de sortie


text_test = "I am Gabriel. My name Melki. I love dogs. I like Mum. I hate brocolli. I play soccer. I eat pasta"

bobo = make_files(text_test,"mon_tenseur.pt","mon_vocab.pt")

V = len(bobo[1].keys()) # taille du dico

cdset = CharDataset("mon_tenseur.pt","mon_vocab.pt",T,n)

dataload = DataLoader(cdset,batch_size=batch_size,shuffle=False)

loss = nn.NLLLoss() #on rentre un logSoftMax

model = RNN(size_x=T, size_y=n, size_dict=V, size_h=size_h, f=F.tanh)
optimizer = torch.optim.Adam(model.parameters(),lr=0.05)
dataload.dataset.data.size()


torch.Size([21])

In [13]:
#APPRENTISSAGE
a = time.time()
i=0

err =Variable(torch.ones(1))
while(time.time() - a < 3*60):
    optimizer.zero_grad()
    err = 0
    for x_train, y_train in dataload:
        mat_x_train = torch.FloatTensor(V, T).fill_(0)
        for j in range(T):
            mat_x_train[x_train[0][j] - 1, j] = 1
        mat_x_train = Variable(mat_x_train)

        y_train = y_train.type(torch.ByteTensor)

        mat_y_train = torch.FloatTensor(V, n).fill_(0)
        for j in range(n):
            mat_y_train[y_train[0][j] - 1, j] = 1
        mat_y_train = Variable(mat_y_train)

        #y_train = Variable(y_train[0])

        h = Variable(torch.zeros(size_h, x_train.size()[0]))

        out = model.forward(h, mat_x_train, mat_y_train)

        for j in range(n):
            err += loss.forward(torch.log(out[j].view(1, V)), Variable(y_train[0])[j].type(torch.LongTensor) )
        if(i%9997==0):
            print("=====================")
            print(err)
            print(code2char(x_train.view(-1),cdset.vocab))
            print(code2char(y_train.view(-1),cdset.vocab))
            print([code2char(torch.max(out[i].data,-1)[1],cdset.vocab) for i in range(n)])            
        i +=1
    err.backward()
    
    optimizer.step()
   



Variable containing:
 8.4215
[torch.FloatTensor of size 1]

IamGabrielMy
nameMelkiI
['Melki', ',', 'I']
Variable containing:
1.00000e-04 *
  3.1162
[torch.FloatTensor of size 1]

lovedogsIlike
MumIhate
['Mum', 'I', 'hate']
Variable containing:
1.00000e-04 *
  2.3896
[torch.FloatTensor of size 1]

brocolliIplaysoccer
Ieatpasta
['I', 'eat', 'pasta']


In [14]:
##INFERENCE
acc=0
k=0
for x_train, y_train in dataload:
    mat_x_train = torch.FloatTensor(V, T).fill_(0)
    for j in range(T):
        mat_x_train[x_train[0][j] - 1, j] = 1
    mat_x_train = Variable(mat_x_train)
    
    mat_y_train = torch.FloatTensor(V, n).fill_(0)
    for j in range(n):
        mat_y_train[y_train[0][j] - 1, j] = 1
    mat_y_train = Variable(mat_y_train)

    out2 = model.forward2(Variable(torch.zeros(size_h, x_train.size()[0])), mat_x_train)
    out = model.forward(Variable(torch.zeros(size_h, x_train.size()[0])), mat_x_train, mat_y_train)
    acc += 1*(out2[0]==y_train[0,0])
    
    k += 1
    if(k%100 == 0):
        print(code2char(x_train.view(-1),cdset.vocab))
        print(code2char(y_train.view(-1),cdset.vocab))
        print(code2char(out2,cdset.vocab))
        print(code2char(torch.max(out[0].data,-1)[1],cdset.vocab))
        print(acc,k)
        print('=============')
print(acc/k)


1.0
